1. Carga y procesamiento de datos .BVH:

Primero, necesitamos cargar y procesar los datos .BVH. Para esto, podemos usar una librería como bvh-python.

Instala la librería bvh-python:

In [1]:
pip install bvh

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#Load packages
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import pickle
import timeit
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import os
import scipy.io as sio
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder
from sklearn.base import  BaseEstimator, TransformerMixin, ClassifierMixin
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
import warnings
warnings.filterwarnings("ignore")

In [3]:

#clase principal shecco
class SHECCO(BaseEstimator,TransformerMixin):
    def __init__(self, sigmax = 54, sigmay = 54, Ax = 1, Ay = 1,ws = 5, noverlap = None, k = 50, sup = 1):
        self.sigmax = sigmax
        self.sigmay = sigmay
        self.ws = ws
        self.noverlap = noverlap
        self.sup = sup
        self.k = k
        self.Ax = Ax
        self.Ay = Ay
        
##################################################################

    def plot_mocap(self, Ns, Nv, ind):
     mp_p = MocapParameterizer(param_type = 'position')
     #print("prueba!")
     #print(list_folder)
     print("NS")
     print(Ns)
     print("list")
     print(list_folder)
     print(Ns)
     data_all = [p.parse('D:/Escritorio/datos_mocap/'+Ns+'/'+list_folder[Nv])]
     pos_d= mp_p.fit_transform(data_all)
     #Normalization
     n=pos_d[0].values
     P = np.asarray(pos_d[0].values)
     print("Shape of P:", P.shape)
     #z = P.reshape(n.shape[0],38,3)
     #z = P.reshape(2999, 38, 3)
     z = P.reshape(2999, 73)
     
     Z = np.zeros(z.shape)
     for i in range(z.shape[0]):
       for j in range(z.shape[1]):
         Z[i,j,:] =z[i,j,:]- z[i,0,:]       
    
     print(f"Shape of ind: {ind.shape}")
     print(f"Shape of Z: {Z.shape}")
    
     #Distance among skeletons
     for k in range(ind.shape[0]):
       for l in range(Z.shape[1]):
         Z[ind[k],l,0]= Z[ind[k],l,0]+20*k
     Z = Z.reshape(n.shape[0],114)
     n.iloc[:]=Z[:]
     pos_d[0].values = n

     if ind.shape[0]<= 10:
       draw_stickfigure(pos_d[0], frame = ind, figsize=(20,4))
     else: 
       nf = int(ind.shape[0]/10)
       for h in range(0,nf):
         draw_stickfigure(pos_d[0], frame = ind[10*h:10*(h+1)], figsize=(20,4))
         plt.show()

########################################

    def geneout_(self,X):
      #print(X.shape)
      N,T = X.shape
      Y = np.zeros((N,T))
      for n in range(N):
        for t in range(T-1):
          Y[n,t]=X[n,t+1]
      return Y

    def ventaneo_(self,X):
       N,P = X.shape
       if self.noverlap == None: self.noverlap =0
       if self.ws == None: self.ws = P
       for j in range(N):
         pi = 0
         pf = self.ws
         Xi = X[j,pi:pf].reshape(1,-1) 
         Yi = X[j,pf].reshape(1,-1)
         pi = pi + (self.ws - self.noverlap)
         pf = pi +self.ws
         while pf  < P:
                Xi = np.r_[Xi,X[j,pi:pf].reshape(1,-1)]
                Yi = np.r_[Yi,X[j,pf].reshape(1,-1)]
                pi = pi + (self.ws - self.noverlap)
                pf = pi +self.ws
                #print(Xi,Yi)
         #print(pi,pf)
         if pf >= P:
            temp = X[j,pi:].reshape(1,-1)
            Xi = np.r_[Xi,np.c_[temp,np.zeros(self.ws-temp.shape[1]).reshape(1,-1)]]
            Yi = np.r_[Yi,np.zeros(1).reshape(-1,1)]       
         if j == 0: 
           #print(Xi.shape)
           X_ = np.zeros((N,Xi.shape[0],Xi.shape[1]))
           Y_ = np.zeros((N,Yi.shape[0],1))
         X_[j] = Xi  
         Y_[j] = Yi  
       return X_,Y_  

    def SC_(self,X,Y):
        Xcl = np.zeros((X.shape[0], self.k))
        Ycl = np.zeros((Y.shape[0], self.k))
        for n in range(X.shape[0]):
            Z = X[n]
            L = Y[n]
            Z = np.tile(Z,(Z.shape[0],1))
            D = pairwise_distances(Z,Z)
            gamma = 1/(np.median(D)**2)
            K = np.exp(-0.5*gamma*D**2)
            Clus = SpectralClustering(n_clusters= self.k,affinity='precomputed')
            y_pred = Clus.fit_predict(K)
            centers = np.zeros((self.k,Z.shape[1]))
            for c in range(self.k):
                centers[c] = Z[y_pred==c].mean(axis=0)
                
            Ds = pairwise_distances(centers,Z)
            ind = np.argmin(Ds,axis=1) 
            Xcl[n] = Z[ind]
            Ycl[n] = L[ind]
        return Xcl,Ycl

    def SC_Mocap(self,X,Y):
      Xcl = np.zeros((self.k, X.shape[1]))
      Ycl = np.zeros((self.k, Y.shape[1]))
      D = pairwise_distances(Y,Y)
      gamma = 1/(np.median(D)**2)
      K = np.exp(-0.5*gamma*D**2)
      Clus = SpectralClustering(n_clusters= self.k,affinity='precomputed')
      y_pred = Clus.fit_predict(K)
      centers = np.zeros((self.k,Y.shape[1]))
      for c in range(self.k):
        centers[c] = Y[y_pred==c].mean(axis=0)

      Ds = pairwise_distances(centers,Y)
      ind = np.argmin(Ds,axis=1)
      Xcl = X[ind]
      Ycl = Y[ind]
      return Xcl,Ycl,ind


    def Kmeans_Mocap(self,X):
      Xcl = np.zeros((self.k, X.shape[1]))
      #Ycl = np.zeros((self.k, Y.shape[1]))
      #Clus = KMeans(n_clusters= self.k, random_state=0)
      y_pred = KMeans(n_clusters= self.k, random_state=0).fit_predict(X)
      centers = np.zeros((self.k,X.shape[1]))
      for c in range(self.k):
        centers[c] = X[y_pred==c].mean(axis=0)

      Ds = pairwise_distances(centers,X)
      ind = np.argmin(Ds,axis=1)
      Xcl = X[ind]
      #Ycl = Y[ind]
      return Xcl,ind

    def kergM(self,X,Y,sigma_,A):
       return A*np.exp(-pairwise_distances(X,Y)**2/(2*sigma_**2))
    
    def shecco_(self,X,Y,Z,L,pxx,pyy,pzz,pll,pxz=None,pyl=None):  
      #se asume independencia
     
      if (pxz==None) & (pyl==None):
          PXX = pxx.reshape(-1,1)#.dot(pxx.reshape(1,-1))
          PYY = pyy.reshape(-1,1)#.dot(pyy.reshape(1,-1))  
          PZZ = pzz.reshape(-1,1)#.dot(pzz.reshape(1,-1))  
          PLL = pll.reshape(-1,1)#.dot(pll.reshape(1,-1))  
          PXY = pxx.reshape(-1,1)#.dot(pyy.reshape(1,-1))  
          PZL = pzz.reshape(-1,1)#.dot(pll.reshape(1,-1))  
      KXX  = self.kergM(X,X,self.sigmax,self.Ax)
      KZZ  = self.kergM(Z,Z,self.sigmax,self.Ax)
      KYY  = self.kergM(Y,Y,self.sigmay,self.Ay)
      KLL  = self.kergM(L,L,self.sigmay,self.Ay)
      KXZ  = self.kergM(X,Z,self.sigmax,self.Ax)
      KYL  = self.kergM(Y,L,self.sigmay,self.Ay)


      #trxx=KXX.shape[0]**2 #np.sqrt(np.trace((KXX.dot(KXX))))
  
      #trzz=KZZ.shape[0]**2#np.sqrt(np.trace((KZZ.dot(KZZ))))
    
      #tryy=KYY.shape[0]**2#np.sqrt(np.trace((KYY.dot(KYY))))
     
      #trll=KLL.shape[0]**2#np.sqrt(np.trace((KLL.dot(KLL))))
    
      #trxz=KXX.shape[0]*KZZ.shape[0]#np.sqrt(np.trace((KXZ.dot(KXZ.T))))
     
      #tryl=KYY.shape[0]*KLL.shape[0]#np.sqrt(np.trace((KYL.dot(KYL.T))))
      #KXX = KXX*trxx
      #KZZ = KZZ*trzz
      #KYY = KYY*tryy
      #KLL = KLL*trll
      #KXZ = KXZ*trxz
      #KYL = KYL*tryl
     
      #d = np.trace((PXX*KXX).dot(KYY*PYY)) -2*np.trace((PXZ*KXZ).dot((KYL*PYL).T)) + np.trace((PZZ*KZZ).dot(KLL*PLL))
      d = (PXY.T).dot(KXX*KYY).dot(PXY) -2*(PXY.T).dot(KXZ*KYL).dot(PZL) + (PZL.T).dot(KZZ*KLL).dot(PZL)

      if d<0:
        d = abs(d)
      return d

    #hecco sin salida (no supervised)
    def hecco_(self,X,Z,pxx,pzz,pxz=None,pyl=None):  
      #se asume independencia
     
      if (pxz==None) & (pyl==None):
          PXX = pxx.reshape(-1,1).dot(pxx.reshape(1,-1))
          PZZ = pzz.reshape(-1,1).dot(pzz.reshape(1,-1))   
          PXZ = pxx.reshape(-1,1).dot(pzz.reshape(1,-1)) 
      KXX  = self.kergM(X,X,self.sigmax)
      KZZ  = self.kergM(Z,Z,self.sigmax)
      KXZ  = self.kergM(X,Z,self.sigmax)
      
      d = np.dot(np.dot(np.ones(PXX.shape[0]).T,(PXX*KXX)),np.ones(PXX.shape[0]))-2*np.dot(np.dot(np.ones(PXX.shape[0]).T,(PXZ*KXZ)),np.ones(PZZ.shape[0]))+ np.dot(np.dot(np.ones(PZZ.shape[0]).T,(PXX*KXX)),np.ones(PZZ.shape[0]))
     
      if d<0:
        d = abs(d)
      return d   

    def SHECCO_M_(self,Xtrain_,Xtest_,Ytrain_,Ytest_,pzz,pll):
      Ntrain = Xtrain_.shape[0]
      Ntest = Xtest_.shape[0]
        
      Dte_tr = np.zeros((Ntest,Ntrain))
      for i in range(Ntrain):
        for j in range(Ntest):
          if self.sup == None:
              Dte_tr[j,i] = self.hecco_(Xtrain_[i],Xtest_[j],self.pxx,pzz)
          else:
            Dte_tr[j,i] = self.shecco_(Xtrain_[i],Ytrain_[i],Xtest_[j],Ytest_[j],
                       self.pxx,self.pyy,pzz,pll)
      return Dte_tr

    def fit(self,X,*_):
        #self.Xtrain_,self.Ytrain_ = self.ventaneo_(X)
        # If we want use Spectral Clustering:
        #if self.k != None:
           # self.Xtrain_, self.Ytrain_= self.SC_(self.Xtrain_,self.Ytrain_)
        self.Xtrain_= X
        Ya = np.zeros((X.shape[0],self.k,114))
        for i in range(X.shape[0]):
          Z = X[i]
          L = dist.geneout_(Z.T)
          Ya[i,:,:]= L.T
        Yp = Ya.reshape(X.shape[0]*X.shape[1],X.shape[2])
        pca = PCA(n_components=3)
        Ytr = pca.fit(Yp).transform(Yp)  
        Y = Ytr.reshape(X.shape[0], X.shape[1], Ytr.shape[1])
        self.Ytrain_= Y
        self.N = self.Xtrain_.shape[1]
        #estimar pdfs
        self.pxx = (1/self.N)*np.ones(self.N) #prob uniformes
        self.pyy = (1/self.N)*np.ones(self.N)
        return self

    def transform(self,X): #Xt_=[Xt,Yt]
        #Xtest_,Ytest_ = self.ventaneo_(X)
        # If we want use Spectral Clustering:
        #if self.k != None:
        Xtest_ = X
        Yb = np.zeros((X.shape[0],self.k,114))
        for i in range(X.shape[0]):
          Z = X[i]
          L = dist.geneout_(Z.T)
          Yb[i,:,:]= L.T
        Yp = Yb.reshape(X.shape[0]*X.shape[1],X.shape[2])
        pca = PCA(n_components=3)
        Yte = pca.fit(Yp).transform(Yp)  
        Y = Yte.reshape(X.shape[0], X.shape[1], Yte.shape[1])
        Ytest_ = Y
        M = Xtest_.shape[1]
        #estimar pdfs
        pzz = (1/M)*np.ones(M) #porbas uniformes
        pll = (1/M)*np.ones(M)
        return self.SHECCO_M_(self.Xtrain_,Xtest_,self.Ytrain_,Ytest_,pzz,pll)

    def fit_tranform(self,X):
        self.fit(X)
        return self.transform(X)

cargar datos bvh por pymo

In [4]:
import os
from bvh import Bvh

# Función para cargar y leer un archivo .BVH
def cargar_bvh(archivo_bvh):
    with open(archivo_bvh, 'r') as f:
        mocap_data = Bvh(f.read())
    return mocap_data

# Directorio que contiene los archivos .BVH
directorio = 'D:/Escritorio/datos_mocap/01'

# Listar todos los archivos en el directorio
archivos_bvh = os.listdir(directorio)

# Filtrar solo los archivos .BVH
archivos_bvh = [archivo for archivo in archivos_bvh if archivo.endswith('.bvh')]

# Ordenar los archivos .BVH
archivos_bvh = sorted(archivos_bvh)

# Mostrar los primeros archivos .BVH para verificar
print(archivos_bvh[:20])
print('******************************************************************************************************')

# Cargar y mostrar información del primer archivo .BVH
if archivos_bvh:
    primer_archivo = archivos_bvh[0]
    ruta_primer_archivo = os.path.join(directorio, primer_archivo)
    datos_mocap = cargar_bvh(ruta_primer_archivo)

    print("Número de frames:", datos_mocap.nframes)
    print("Frame time:", datos_mocap.frame_time)
    print("Joints:", datos_mocap.get_joints())
else:
    print("No se encontraron archivos .BVH en el directorio.")



['adorozco_Derecha.bvh', 'adorozco_Remate.bvh', 'adorozco_Reves.bvh', 'adorozco_Servicio.bvh', 'adorozco_VDerecha.bvh', 'adorozco_VReves.bvh']
******************************************************************************************************
Número de frames: 2999
Frame time: 0.01
Joints: [ROOT Hips, JOINT Chest, JOINT Neck, JOINT Head, JOINT LeftCollar, JOINT LeftShoulder, JOINT LeftElbow, JOINT LeftWrist, JOINT RightCollar, JOINT RightShoulder, JOINT RightElbow, JOINT RightWrist, JOINT LeftHip, JOINT LeftKnee, JOINT LeftAnkle, JOINT RightHip, JOINT RightKnee, JOINT RightAnkle]


2. Análisis de datos
Podemos hacer algunos análisis básicos, como extraer las posiciones de los joints a lo largo del tiempo.

Primero, imprimamos la estructura jerárquica del archivo BVH para entender mejor su contenido.

In [5]:
# Función para imprimir la estructura jerárquica del archivo BVH
def imprimir_estructura_bvh(bvh_data):
    def imprimir_joint(joint, nivel=0):
        joint_name = joint['JOINT'][0] if 'JOINT' in joint else joint['ROOT'][0]
        print('  ' * nivel + joint_name)
        if 'CHANNELS' in joint:
            for channel in joint['CHANNELS'][1:]:
                print('  ' * (nivel + 1) + 'CHANNEL: ' + channel)
        if 'children' in joint:
            for child_joint in joint['children']:
                imprimir_joint(child_joint, nivel + 1)

    root_joint = bvh_data.root
    imprimir_joint(root_joint)

# Imprimir la estructura del archivo BVH
imprimir_estructura_bvh(datos_mocap)

Hips


3. Visualización de los datos
Para la visualización, podemos usar matplotlib y mpl_toolkits.mplot3d.

In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Función para visualizar los joints en un frame específico
def visualizar_frame(df, frame_num):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    for joint in datos_mocap.get_joints():
        x = df.loc[frame_num, f'{joint}_X']
        y = df.loc[frame_num, f'{joint}_Y']
        z = df.loc[frame_num, f'{joint}_Z']
        ax.scatter(x, y, z, label=joint)
    
    ax.legend()
    plt.show()

# Visualizar el primer frame
visualizar_frame(df_mocap, 0)


NameError: name 'df_mocap' is not defined

4. Preparación para Machine Learning
Antes de entrenar un modelo de machine learning, necesitamos preparar los datos. Esto puede incluir normalización, división en conjunto de entrenamiento y prueba, y más.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Normalizar los datos
scaler = StandardScaler()
df_mocap_scaled = scaler.fit_transform(df_mocap)

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test = train_test_split(df_mocap_scaled, test_size=0.2, random_state=42)

# Mostrar las dimensiones de los conjuntos
print("Dimensiones del conjunto de entrenamiento:", X_train.shape)
print("Dimensiones del conjunto de prueba:", X_test.shape)


************************PRUEBA